In [1]:
import numpy as np
import pandas as pd

In [ ]:
# model_name = "danielribeiro/google-play-sentiment-analysis"
# Use a pipeline as a high-level helper

# from transformers import pipeline
# pipe = pipeline("text-classification", model="danielribeiro/google-play-sentiment-analysis")
# pipe('qyakqerl')

# SST2 - PT

In [31]:
from datasets import load_dataset

ds = load_dataset("maritaca-ai/sst2_pt")

In [32]:
data_val = ds['validation'].rename_column('label', 'true_label')

In [33]:
df_sst2 = pd.DataFrame(data=data_val)

In [34]:
df_sst2.head()

,text,true_label
0,É uma jornada encantadora e muitas vezes afeta.,1
1,inabalável e desesperado,0
2,Permite -nos esperar que Nolan esteja pronto p...,1
3,"A atuação, figurinos, música, cinematografia e...",1
4,"É lento - muito, muito lento.",0


In [7]:
# df_sst2.to_csv('../data/maritaca-ai_sst2_pt.csv', index=False)

In [8]:
# ds = load_dataset('csv', data_files='../data/maritaca-ai_sst2_pt.csv')

Generating train split: 872 examples [00:00, 45908.43 examples/s]


# IMDB - PT

In [3]:
from datasets import load_dataset

ds = load_dataset("maritaca-ai/imdb_pt")

/home/viviane/Documents/Computação Evolutiva/Trabalho_2/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test_all split: 100%|██████████| 25000/25000 [00:00<00:00, 135495.59 examples/s]


In [4]:
imdb_val = ds['test'].rename_column('label', 'true_label')

In [5]:
df_imdb = pd.DataFrame(data=imdb_val)

In [6]:
df_imdb.head()

,text,true_label
0,Como coisas assim ainda podem ser feitas? Sein...,0
1,Half -Have Dead era diferente de qualquer film...,0
2,"Outro filme do país montanhoso sueco, que é se...",0
3,"OK, eu fui a este site antes de assistir a est...",0
4,Esta é de longe a pior coisa que já vi no film...,0


In [7]:
df_imdb.to_csv('../data/maritaca-ai_imdb_pt.csv', index=False)

# Concatena as bases

In [39]:
df = pd.concat([df_sst2, df_imdb], axis=0)

In [40]:
df.head()

,text,true_label
0,É uma jornada encantadora e muitas vezes afeta.,1
1,inabalável e desesperado,0
2,Permite -nos esperar que Nolan esteja pronto p...,1
3,"A atuação, figurinos, música, cinematografia e...",1
4,"É lento - muito, muito lento.",0


In [42]:
df_imdb.shape, df_sst2.shape, df.shape

((5000, 2), (872, 2), (5872, 2))

In [43]:
df.to_csv('../data/maritaca-ai_sst2_imdb_pt.csv', index=False)

In [12]:
# vals = data_val.map(lambda x: pipe(x['text'])[0])
# vals[0]
# df = pd.DataFrame(vals)
# df.head()
# df['model_label'] = df['label'].replace('Positivo', 1).replace('Negativo', 0).replace('Neutro', -1)
# df[df['label']!= 'Neutro']['model_label'].value_counts()
# from sklearn.metrics import classification_report

# print(classification_report(
#     df[df['label']!='Neutro']['true_label'], 
#     df[df['label']!='Neutro']['model_label']))
# def objective_function(x, *args):
#     """created from `parallel_objective` argument"""
#     return parallel_objective([x], *args)[0]

In [8]:
import pandas as pd
from sklearn.metrics import f1_score
from transformers import pipeline
from datasets import load_dataset

def _eval_model(merged_path: str, dataset: str):
    
    print(f'Avaliando modelo {merged_path}')
    
    pipe = pipeline(
        "text-classification", 
        model=merged_path,
        tokenizer=merged_path,
        device='cuda',
        truncation=True
    )
    tokenizer_kwargs = {
        'padding':True,
        'truncation':True,
        'max_length':512
    }

    data_val = load_dataset('csv', data_files=dataset)
    vals = data_val['train'].map(
        lambda x: pipe(x['text'], **tokenizer_kwargs)[0]
    )
    df = pd.DataFrame(vals)
    df['model_label'] = df['label'].replace('Positivo', 1).replace('Negativo', 0).replace('Neutro', -1)
    res = f1_score(
        df[df['label']!='Neutro']['true_label'], 
        df[df['label']!='Neutro']['model_label'], 
        average='binary'
    )
    results = {
        'dataset': {
            'acc,none': res,
        }
    }
    return {"score": res, "results": results}

In [9]:
model_name = "danielribeiro/google-play-sentiment-analysis"
_eval_model(model_name, '../data/maritaca-ai_sst2_pt.csv')

Avaliando modelo danielribeiro/google-play-sentiment-analysis


Map: 100%|██████████| 872/872 [00:29<00:00, 29.67 examples/s]
/tmp/ipykernel_377494/834985490.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['model_label'] = df['label'].replace('Positivo', 1).replace('Negativo', 0).replace('Neutro', -1)


{'score': 0.8701298701298701,
 'results': {'dataset': {'acc,none': 0.8701298701298701}}}

In [10]:
model_name = "danielribeiro/google-play-sentiment-analysis"
_eval_model(model_name, '../data/maritaca-ai_imdb_pt.csv')

Avaliando modelo danielribeiro/google-play-sentiment-analysis


Map: 100%|██████████| 5000/5000 [11:12<00:00,  7.44 examples/s]
/tmp/ipykernel_377494/834985490.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['model_label'] = df['label'].replace('Positivo', 1).replace('Negativo', 0).replace('Neutro', -1)


{'score': 0.7833643699565487,
 'results': {'dataset': {'acc,none': 0.7833643699565487}}}